### Finding the intersect of pythons ranges

I have a lot data in paginated memory blocks. Each page is generally of a block size but can vary due to inserts and deletes.

My challenge: Determine whether the contents of a page needs to be loaded to answer a users query for slice.

Here's a simplistic example:

```
page = range(500,700,1)
user = range(10,1000,30)
userdata = []
for row in intercept(page,user):
      userdata.append(row)
```

Here is the solution:

In [1]:
import math

In [2]:
def intercept(A, B):
    """Enables calculation of the intercept of two range objects.
    Used to determine if a datablock contains a slice.

    Args:
        A: range
        B: range

    Returns:
        range: The intercept of ranges A and B.
    """
    if A.step < 1:
        A = range(A.stop + 1, A.start + 1, 1)
    if B.step < 1:
        B = range(B.stop + 1, B.start + 1, 1)

    if len(A) == 0:
        return range(0)
    if len(B) == 0:
        return range(0)

    if A.stop <= B.start:
        return range(0)
    if A.start >= B.stop:
        return range(0)

    if A.start <= B.start:
        if A.stop <= B.stop:
            start, end = B.start, A.stop
        elif A.stop > B.stop:
            start, end = B.start, B.stop
        else:
            raise ValueError("bad logic")
    elif A.start < B.stop:
        if A.stop <= B.stop:
            start, end = A.start, A.stop
        elif A.stop > B.stop:
            start, end = A.start, B.stop
        else:
            raise ValueError("bad logic")
    else:
        raise ValueError("bad logic")

    a_steps = math.ceil((start - A.start) / A.step)
    a_start = (a_steps * A.step) + A.start

    b_steps = math.ceil((start - B.start) / B.step)
    b_start = (b_steps * B.step) + B.start

    if A.step == 1 or B.step == 1:
        start = max(a_start, b_start)
        step = max(A.step, B.step)
        return range(start, end, step)
    elif A.step == B.step:
        a, b = min(A.start, B.start), max(A.start, B.start)
        if (b - a) % A.step != 0:  # then the ranges are offset.
            return range(0)
        else:
            return range(b, end, step)
    else:
        # determine common step size:
        step = max(A.step, B.step) if math.gcd(A.step, B.step) != 1 else A.step * B.step
        # examples:
        # 119 <-- 17 if 1 != 1 else 119 <-- max(7, 17) if math.gcd(7, 17) != 1 else 7 * 17
        #  30 <-- 30 if 3 != 1 else 90 <-- max(3, 30) if math.gcd(3, 30) != 1 else 3*30
        if A.step < B.step:
            for n in range(a_start, end, A.step):  # increment in smallest step to identify the first common value.
                if n < b_start:
                    continue
                elif (n - b_start) % B.step == 0:
                    return range(n, end, step)  # common value found.
        else:
            for n in range(b_start, end, B.step):
                if n < a_start:
                    continue
                elif (n - a_start) % A.step == 0:
                    return range(n, end, step)

        return range(0)

In [3]:
A = range(500,700,3)
B = range(520,700,3)
C = range(10,1000,30)

assert intercept(A,C) == range(0)
assert set(intercept(B,C)) == set(B).intersection(set(C))

A = range(500_000, 700_000, 1)
B = range(10, 10_000_000, 1000)

assert set(intercept(A,B)) == set(A).intersection(set(B))

A = range(500_000, 700_000, 1)
B = range(10, 10_000_000, 1)

assert set(intercept(A,B)) == set(A).intersection(set(B))

In [4]:
A = range(500_000, 700_000, 1)
B = range(10, 10_000_000, 1000)

In [5]:
%timeit intercept(A,B)

834 ns ± 12.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


That's fast enough for me.